In [ ]:
!pip install git+https://github.com/PrithivirajDamodaran/Styleformer.git

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import csv
from styleformer import Styleformer
import torch
import warnings
warnings.filterwarnings("ignore")

#uncomment for re-producability
def set_seed(seed):
  torch.manual_seed(seed)
  if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

set_seed(1234)


# style = [0=Casual to Formal, 1=Formal to Casual, 2=Active to Passive, 3=Passive to Active etc..]
sf_zero = Styleformer(style = 0)

In [ ]:
data = pd.read_csv("Data.csv")
# Input File: Data.csv >> Columns: category_slug; asin; question; answer;
cat_slugs = list()
asins = list()
questions = list()
answers = list()
questions_sqcf = list()
answers_sacf = list()
asin_count = 0
prev_asin = None
for i,r in data.iterrows():
  if prev_asin is None:
    prev_asin = r['asin']
  if asin_count > 9:
    if prev_asin is not r['asin']:
      asin_count = 0
    else:
      asin_count = asin_count + 1
      continue
  elif prev_asin is not r['asin']:
    asin_count = 0
  tsq_formal = sf_zero.transfer(r['question'])
  tsa_formal = sf_zero.transfer(r['answer'])
  cat_slugs.append(r['category_slug'])
  asins.append(r['asin'])
  questions.append(r['question'])
  answers.append(r['answer'])
  questions_sqcf.append(tsq_formal)
  answers_sacf.append(tsa_formal)
  prev_asin = r['asin']
  asin_count = asin_count + 1
  print(r['asin'], ">> question #", asin_count)
  print("line #", i+1)
  df = pd.DataFrame()
  df['category'] = cat_slugs
  df['asin'] = asins
  df['question'] = questions
  df['answer'] = answers
  df['Style question casual to formal'] = questions_sqcf
  df['Style answer casual to formal'] = answers_sacf

  df.to_csv('/content/drive/MyDrive/Styleformer/style_Data.csv', index=False)
  # Output File: style_Data.csv >> Columns: category_slug; asin; question; answer; Style question casual to formal; Style answer casual to formal;